# Minimum Snap Trajectory
## Model
* control input $\mathbf u = [u_1, u_2, u_3, u_4]^T$
	* $u_1$ net body force
	* $u_2, u_3, u_4$ body moments according to rotor speeds
* state $\mathbf x = [x, y, z, \phi, \theta, \psi, \dot{x}, \dot{y}, \dot{z}, p, q, r]^T$

## Differential Flatness
* flat outputs $\sigma = [x, y, z , \psi]^T$
	* coordinates of COM $\mathbf r = [x, y, z]^T$
	* yaw angle $\psi$


## Trajectory Optimization
Quadratic Program (QP):
$$
\min\int_{t_o}^{t_m}
\mu_r \left|\left|\frac{d^{k_r}\mathbf{r}_{\text{T}}}{dt^{k_r}}\right|\right| +
\mu_\psi \left|\left|\frac{d^{k_\psi}\psi_{\text{T}}}{dt^{k_\psi}}\right|\right|
dt
$$
s.t.
$$
\begin{aligned}
\sigma_T(t_i)&=\sigma_i ,&\qquad j = 0, \dots, m \\
\left.\frac{d^Px_T}{dt^P}\right|_{t=t_j} &= 0 \text{ or free,} &\qquad  j = 0, \dots, m; p=1, \dots, k_r \\
\left.\frac{d^Py_T}{dt^P}\right|_{t=t_j} &= 0 \text{ or free,} &\qquad  j = 0, \dots, m; p=1, \dots, k_r \\
\left.\frac{d^Pz_T}{dt^P}\right|_{t=t_j} &= 0 \text{ or free,} &\qquad  j = 0, \dots, m; p=1, \dots, k_r \\
\left.\frac{d^P\psi_T}{dt^P}\right|_{t=t_j} &= 0 \text{ or free,} &\qquad  j = 0, \dots, m; p=1, \dots, k_\psi
\end{aligned}
$$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys, rospy, rosbag, glob, re, math

import scipy
import scipy.stats
from scipy.stats import pearsonr

from smap.msg import smapStats

from IPython.display import HTML, display

from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib import cm
from numpy import linspace
from matplotlib.colors import LogNorm
from decimal import *

def deg2rad(deg):
    return deg / 180. * np.pi

keyframes = [
    [0, 0, deg2rad(0)],
    [2, 0, deg2rad(90)],
    [2, 6, deg2rad(180)],
    [0, 6, deg2rad(0)]
]
times = [0, 0.3, 0.7, 1.0]

## Nondimensionalization
Quantities $x_t, y_T, z_T, \psi_T$ are decoupled in cost function and constraints of the QP.
Hence, they can be separated into 4 optimization problems. Considering a general form for a nondimensional variable $\tilde{w}(\tau)$ where $\tau$ represents nondimensionalized time:
$$
\begin{aligned}
\min &\int_0^1 \frac{d^k\tilde{w}(\tau)^2}{d\tau^k}d\tau&\\
\text{s.t.} &\qquad \tilde{w}(\tau_i) = \tilde{w}_i, &\qquad i=0,\dots,m\\
&\qquad \left.\frac{d^P\tilde{w}(\tau)}{d\tilde{w}^P}\right|_{\tau=\tau_j}=0\text{ or free}, &\qquad t_j=0,\dots,1; p=1,\dots,k
\end{aligned}
$$

In [3]:
k = 3 # dimensions
d = 5 # polynomial degree

C = [
    [0, 0, 1],
    [1, 0, 0]
]

times = np.linspace(0, 1, 100)

def polynomial(times, constraints, continuity=2, degree=4):
    pass

### Evaluation for a single polynomial
$$
\begin{aligned}
